In [1]:
!which python

/Users/pushkalm11/miniconda3/envs/unrolling/bin/python


In [2]:
import matplotlib.pyplot as plt
import numpy as np
from numpy import linalg as LA
from numpy.linalg import matrix_power as MP

In [3]:
import random
import scipy
from scipy.spatial.distance import cdist
from scipy.io import savemat, loadmat

In [4]:
import networkx as nx
import sys

In [5]:
import tensorflow as tf
from tensorflow.keras import initializers
from tensorflow.keras.layers import Flatten, Dense
from keras import backend as K
from keras.callbacks import EarlyStopping

In [6]:
print(f"TF Version: {tf.__version__}")

TF Version: 2.14.0


In [7]:
X_train = loadmat('./Dataset/X_train.mat')['X']
Adj_matrix = loadmat('./Dataset/Chepuri.mat')['A']

N, T = X_train.shape

In [8]:
dt = np.zeros([T - 1, T])

for i in range(T - 1):
    dt[i, i] = -1
    dt[i, i - 1] = 1

d = dt.T
d = np.concatenate((d, np.zeros([T, 1])), axis = 1) 
d[T - 1, T - 1] = -1
d = d.T
d[T - 1, T - 1] = 1 
d[-1, -2] = 1
d[0, -1] = 0
d = tf.convert_to_tensor(d, dtype = "float32")
print(d)

tf.Tensor(
[[-1.  0.  0. ...  0.  0.  0.]
 [ 1. -1.  0. ...  0.  0.  0.]
 [ 0.  1. -1. ...  0.  0.  0.]
 ...
 [ 0.  0.  0. ... -1.  0.  0.]
 [ 0.  0.  0. ...  1. -1.  0.]
 [ 0.  0.  0. ...  0.  1.  1.]], shape=(152, 152), dtype=float32)


In [9]:
def adj_to_laplacian(AdjL):
    tmp = np.array(AdjL, dtype = "float32")
    return np.diag(np.sum(tmp, axis = 0)) - tmp

In [10]:
psi_cost = 0
test_list = []

In [11]:
# A - Predicted, B - Target matrix
def costfunc(A, B):
    # (y_true, y_pred)
    # print(A.shape)
    # print(B.shape)
    return (tf.norm(tf.multiply(psi_cost, (A - B)))) ** 2

# def costfunc(A, B):
#     return (tf.norm(tf.multiply(psi_cost, (A - B))) ** 2) + (reg * tf.norm(HLG @ B @ GLT) ** 2 / 2)

def mse_unknown(A, B):

    global psi_cost
    psi = psi_cost

    psid = tf.convert_to_tensor(np.ones([N, T], dtype = float) - psi, dtype = "float32")
    mse_unk = (LA.norm(np.multiply(psid, (X_train - B)))) ** 2 / np.sum(psid.numpy().flatten())
    
    global mse_unknown_list
    mse_unknown_list.append(mse_unk)
    
    return mse_unk

def mse_known(A, B):

    global psi_cost 
    psi = psi_cost

    mse_kn = (LA.norm(np.multiply(psi, (X_train - B)))) ** 2 / np.sum(psi.numpy().flatten())
    
    global mse_known_list
    mse_known_list.append(mse_kn)

    return mse_kn

In [12]:
callb = EarlyStopping(monitor = 'loss', mode = 'min', min_delta = 0.00001, verbose = 1, patience = 10)

def scheduler(epoch, lr):
    if epoch < 10:
        return lr
    else:
        return lr / lr_decay_factor

lr_scheduler = tf.keras.callbacks.LearningRateScheduler(scheduler)

In [13]:
class EldarModel(tf.keras.Model):
    
    def __init__(self, Laplacian, reg_init, loops_init, d_init, **kwargs):
        super(EldarModel, self).__init__(**kwargs)
        p_init = 20.0
        self.p1_init = tf.constant(p_init, shape = (1, 1), dtype = "float32")
        self.p1 = tf.Variable(initial_value = self.p1_init, trainable = True, name = "p1")
        self.p2_init = tf.constant(p_init, shape = (1, 1), dtype = "float32")
        self.p2 = tf.Variable(initial_value = self.p2_init, trainable = True, name = "p2")
        self.p3_init = tf.constant(p_init, shape = (1, 1), dtype = "float32")
        self.p3 = tf.Variable(initial_value = self.p3_init, trainable = True, name = "p3")
        self.p4_init = tf.constant(p_init, shape = (1, 1), dtype = "float32")
        self.p4 = tf.Variable(initial_value = self.p4_init, trainable = True, name = "p4")
        
        self.lap = Laplacian
        self.reg = reg_init
        self.loops = loops_init
        self.d_matrix = d_init
        
    def call(self, y):
        
        psi = y[0, :, T:]
        global psi_cost
        psi_cost = psi
        
        psi = tf.convert_to_tensor(psi, dtype = "float32")
        
        X_in = y[0, :, : T]
        X_in = tf.convert_to_tensor(X_in, dtype = "float32")
        
        Y = X_in
        
        LG, reg, loop, d = self.lap, self.reg, self.loops, self.d_matrix
        
        LT = tf.matmul(tf.transpose(d), d)
        LT = tf.convert_to_tensor(LT, dtype = "float32")
        
        HLG = tf.eye(tf.shape(LG)[0], dtype = "float32") 
        GLT = LT + (self.p1 * MP(LT, 2)) + (self.p2 * MP(LT, 3)) + (self.p3 * MP(LT, 4)) + (self.p4 * MP(LT, 5))
        
        Xk = tf.zeros_like(X_in)
        Zk = tf.zeros_like(X_in)
        Zk = -(tf.multiply(psi, Xk) - Y + (reg * (HLG @ Xk @ GLT)))
        
        for i in range(loop):
            fdx_xk = tf.multiply(psi, Xk) - Y + (reg * (HLG @ Xk @ GLT))
            fdx_zk = tf.multiply(psi, Zk) - Y + (reg * (HLG @ Zk @ GLT))

            # row = 4
            # a = (tf.multiply(psi, Xk) - Y).numpy()[:,row]
            # b = (reg * (HLG @ Xk @ GLT)).numpy()[:,row]
            # for k in range(len(a)):
            #     print(a[k], b[k])
            # print("\n")
            
            tau = tf.linalg.trace(tf.transpose(fdx_xk) @ Zk) / tf.linalg.trace(tf.transpose((Y + fdx_zk)) @ Zk)
            
            Xk_1 = Xk - (tau * Zk)
            
            fdx_xk_1 = tf.multiply(psi, Xk_1) - Y + (reg * (HLG @ Xk_1 @ GLT))
            
            gamma = (tf.norm(fdx_xk_1) ** 2) / (tf.norm(fdx_xk) ** 2)
            
            Zk_1 = (gamma * Zk) - fdx_xk_1
            
            Xk = Xk_1
            Zk = Zk_1
        
        global test_list
        test_list.append(Xk_1.numpy())
        
        return tf.reshape(Xk_1, [1, N, T])

In [14]:
no_of_psi = 20
Laplacian = adj_to_laplacian(Adj_matrix)

sensing_ratio = [0.01, 0.05, 0.10, 0.15, 0.20, 0.25, 0.30, 0.35, 0.4, 0.45, 0.5]

# reg_list = np.load("./Outputs/eldar_GT_reg.npy")
# loops_list = np.load("./Outputs/eldar_GT_loops.npy")

reg_list = [1.0e-5] * len(sensing_ratio)
loops_list = [50] * len(sensing_ratio)

all_psi = []

mse_known_list = []
mse_unknown_list = []

learned_variables = []

lr_decay_factor = 1.01

In [15]:
for i_sen, rem in enumerate(sensing_ratio):
    
    print(f"\n\n###########################################################################################")
    print(f"################################### Sensing Ratio: {rem * 100}% ###################################")
    print(f"###########################################################################################\n\n")
    
    if i_sen == 0:
        M = int(rem * T)
        X_train_missing = np.zeros([no_of_psi, N, T])
        X_train_concatenated = np.zeros([no_of_psi, N, 2*T])
    
        for i in range(no_of_psi):
            psi_k = np.array([0] * (N * M) + [1] * (N * (T - M)))
            np.random.shuffle(psi_k)
            psi_k = psi_k.reshape([N, T])
            all_psi.append(psi_k)
            X_train_missing[i, : , : ] = X_train * psi_k
            X_train_concatenated[i, :, :] = np.concatenate((X_train_missing[i], psi_k), axis = 1)
    
    else:
        previous_psi = all_psi[-no_of_psi : ]
        X_train_missing = np.zeros([no_of_psi, N, T])
        X_train_concatenated = np.zeros([no_of_psi, N, 2 * T])
        
        rem_diff = int(N * (rem - sensing_ratio[i_sen - 1]) * T)
        
        for i in range(no_of_psi):
            psi_k_tmp = np.array(previous_psi[i]).flatten()
            ones_idx = np.where(psi_k_tmp == 1)
            
            ones_to_zero_idx = np.random.choice(ones_idx[0], size = rem_diff)
            psi_k_tmp[ones_to_zero_idx] = 0
            psi_k = psi_k_tmp
            
            psi_k = psi_k.reshape([N, T])
            all_psi.append(psi_k)
            X_train_missing[i, : , : ] = X_train * psi_k
            X_train_concatenated[i, :, :] = np.concatenate((X_train_missing[i], psi_k), axis = 1)
        
    reg_init = reg_list[i_sen]
    loops_init = loops_list[i_sen]
    
    sgd_optimizer = tf.keras.optimizers.SGD(learning_rate = 1.0e-3, momentum = 0.05)
    
    eldar_model = EldarModel(Laplacian, reg_init, loops_init, d, name = 'EM')
    eldar_model.compile(optimizer = sgd_optimizer, loss = costfunc, 
                        metrics = [mse_known, mse_unknown], run_eagerly = True)
    eldar_model.fit(X_train_concatenated, X_train_missing, epochs = 100, 
                    callbacks = [lr_scheduler], batch_size = 1)
    eldar_model.summary()
    
    all_variables = eldar_model.variables
    
    graph_vars = [i.numpy()[0, 0] for i in all_variables]
    learned_variables.append(graph_vars)
    
    print(f"Learned Variables: {graph_vars}")



###########################################################################################
################################### Sensing Ratio: 1.0% ###################################
###########################################################################################


Epoch 1/100
20/20 [==============================] - 7s 365ms/step - loss: 5.6542 - mse_known: 0.0012 - mse_unknown: 0.4805 - lr: 0.0010
Epoch 2/100
20/20 [==============================] - 7s 358ms/step - loss: 5.6515 - mse_known: 0.0012 - mse_unknown: 0.4805 - lr: 0.0010
Epoch 3/100
20/20 [==============================] - 7s 359ms/step - loss: 5.6489 - mse_known: 0.0012 - mse_unknown: 0.4805 - lr: 0.0010
Epoch 4/100
20/20 [==============================] - 7s 370ms/step - loss: 5.6462 - mse_known: 0.0012 - mse_unknown: 0.4805 - lr: 0.0010
Epoch 5/100
20/20 [==============================] - 8s 376ms/step - loss: 5.6435 - mse_known: 0.0012 - mse_unknown: 0.4805 - lr: 0.0010
Epoch 6/100
20/20 [================

Learned Variables: [19.989794, 19.96376, 19.86665, 19.50292]


###########################################################################################
################################### Sensing Ratio: 5.0% ###################################
###########################################################################################


Epoch 1/100
20/20 [==============================] - 7s 359ms/step - loss: 4.9291 - mse_known: 0.0011 - mse_unknown: 0.4031 - lr: 0.0010
Epoch 2/100
20/20 [==============================] - 7s 357ms/step - loss: 4.9271 - mse_known: 0.0011 - mse_unknown: 0.4031 - lr: 0.0010
Epoch 3/100
20/20 [==============================] - 7s 358ms/step - loss: 4.9250 - mse_known: 0.0011 - mse_unknown: 0.4031 - lr: 0.0010
Epoch 4/100
20/20 [==============================] - 7s 359ms/step - loss: 4.9230 - mse_known: 0.0011 - mse_unknown: 0.4031 - lr: 0.0010
Epoch 5/100
20/20 [==============================] - 7s 358ms/step - loss: 4.9210 - mse_known: 0.0011 - mse_unk

Learned Variables: [19.991055, 19.967804, 19.88245, 19.565823]


###########################################################################################
################################### Sensing Ratio: 10.0% ###################################
###########################################################################################


Epoch 1/100
20/20 [==============================] - 8s 417ms/step - loss: 4.1292 - mse_known: 9.3732e-04 - mse_unknown: 0.4229 - lr: 0.0010
Epoch 2/100
20/20 [==============================] - 8s 402ms/step - loss: 4.1278 - mse_known: 9.3699e-04 - mse_unknown: 0.4229 - lr: 0.0010
Epoch 3/100
20/20 [==============================] - 8s 407ms/step - loss: 4.1264 - mse_known: 9.3667e-04 - mse_unknown: 0.4229 - lr: 0.0010
Epoch 4/100
20/20 [==============================] - 8s 380ms/step - loss: 4.1249 - mse_known: 9.3634e-04 - mse_unknown: 0.4229 - lr: 0.0010
Epoch 5/100
20/20 [==============================] - 8s 385ms/step - loss: 4.1235 - mse_know

Learned Variables: [19.992077, 19.97209, 19.90004, 19.635563]


###########################################################################################
################################### Sensing Ratio: 15.0% ###################################
###########################################################################################


Epoch 1/100
20/20 [==============================] - 7s 362ms/step - loss: 3.3527 - mse_known: 8.0440e-04 - mse_unknown: 0.4485 - lr: 0.0010
Epoch 2/100
20/20 [==============================] - 7s 364ms/step - loss: 3.3518 - mse_known: 8.0417e-04 - mse_unknown: 0.4485 - lr: 0.0010
Epoch 3/100
20/20 [==============================] - 7s 369ms/step - loss: 3.3508 - mse_known: 8.0395e-04 - mse_unknown: 0.4485 - lr: 0.0010
Epoch 4/100
20/20 [==============================] - 7s 358ms/step - loss: 3.3499 - mse_known: 8.0372e-04 - mse_unknown: 0.4485 - lr: 0.0010
Epoch 5/100
20/20 [==============================] - 7s 368ms/step - loss: 3.3489 - mse_known

Learned Variables: [19.993273, 19.976612, 19.917591, 19.703789]


###########################################################################################
################################### Sensing Ratio: 20.0% ###################################
###########################################################################################


Epoch 1/100
20/20 [==============================] - 8s 397ms/step - loss: 2.7183 - mse_known: 6.9116e-04 - mse_unknown: 0.4890 - lr: 0.0010
Epoch 2/100
20/20 [==============================] - 8s 396ms/step - loss: 2.7177 - mse_known: 6.9100e-04 - mse_unknown: 0.4890 - lr: 0.0010
Epoch 3/100
20/20 [==============================] - 8s 400ms/step - loss: 2.7171 - mse_known: 6.9084e-04 - mse_unknown: 0.4890 - lr: 0.0010
Epoch 4/100
20/20 [==============================] - 8s 399ms/step - loss: 2.7165 - mse_known: 6.9068e-04 - mse_unknown: 0.4890 - lr: 0.0010
Epoch 5/100
20/20 [==============================] - 8s 419ms/step - loss: 2.7158 - mse_kno

...

Epoch 71/100
10/10 [==============================] - 8s 837ms/step - loss: 718.1785 - mse_known: 0.1174 - mse_unknown: 0.1697 - lr: 9.1404e-05
Epoch 72/100
10/10 [==============================] - 8s 827ms/step - loss: 718.0034 - mse_known: 0.1174 - mse_unknown: 0.1697 - lr: 8.7889e-05
Epoch 73/100
10/10 [==============================] - 8s 842ms/step - loss: 717.8549 - mse_known: 0.1174 - mse_unknown: 0.1696 - lr: 8.4508e-05
Epoch 74/100
10/10 [==============================] - 9s 838ms/step - loss: 717.6407 - mse_known: 0.1173 - mse_unknown: 0.1696 - lr: 8.1258e-05
Epoch 75/100
10/10 [==============================] - 9s 863ms/step - loss: 717.5051 - mse_known: 0.1173 - mse_unknown: 0.1696 - lr: 7.8133e-05
Epoch 76/100
10/10 [==============================] - 8s 845ms/step - loss: 717.3964 - mse_known: 0.1173 - mse_unknown: 0.1696 - lr: 7.5128e-05
Epoch 77/100
10/10 [==============================] - 8s 832ms/step - loss: 717.3125 - mse_known: 0.1173 - mse_unknown: 0.1696 - lr: 7.2

10/10 [==============================] - 8s 850ms/step - loss: 703.5638 - mse_known: 0.1248 - mse_unknown: 0.1783 - lr: 6.0057e-04
Epoch 24/100
10/10 [==============================] - 8s 833ms/step - loss: 702.6227 - mse_known: 0.1246 - mse_unknown: 0.1782 - lr: 5.7748e-04
Epoch 25/100
10/10 [==============================] - 9s 848ms/step - loss: 701.8136 - mse_known: 0.1245 - mse_unknown: 0.1781 - lr: 5.5526e-04
Epoch 26/100
10/10 [==============================] - 8s 839ms/step - loss: 701.0501 - mse_known: 0.1244 - mse_unknown: 0.1780 - lr: 5.3391e-04
Epoch 27/100
10/10 [==============================] - 8s 846ms/step - loss: 700.2059 - mse_known: 0.1242 - mse_unknown: 0.1779 - lr: 5.1337e-04
Epoch 28/100
10/10 [==============================] - 9s 870ms/step - loss: 699.4933 - mse_known: 0.1241 - mse_unknown: 0.1779 - lr: 4.9363e-04
Epoch 29/100
10/10 [==============================] - 9s 850ms/step - loss: 698.7310 - mse_known: 0.1240 - mse_unknown: 0.1778 - lr: 4.7464e-04
Epoch

Epoch 80/100
10/10 [==============================] - 8s 839ms/step - loss: 682.4371 - mse_known: 0.1211 - mse_unknown: 0.1759 - lr: 6.4219e-05
Epoch 81/100
10/10 [==============================] - 8s 833ms/step - loss: 682.3782 - mse_known: 0.1211 - mse_unknown: 0.1758 - lr: 6.1749e-05
Epoch 82/100
10/10 [==============================] - 8s 842ms/step - loss: 682.2139 - mse_known: 0.1210 - mse_unknown: 0.1758 - lr: 5.9374e-05
Epoch 83/100
10/10 [==============================] - 8s 843ms/step - loss: 682.1579 - mse_known: 0.1210 - mse_unknown: 0.1758 - lr: 5.7091e-05
Epoch 84/100
10/10 [==============================] - 8s 827ms/step - loss: 682.0883 - mse_known: 0.1210 - mse_unknown: 0.1758 - lr: 5.4895e-05
Epoch 85/100
10/10 [==============================] - 8s 849ms/step - loss: 681.9774 - mse_known: 0.1210 - mse_unknown: 0.1758 - lr: 5.2784e-05
Epoch 86/100
10/10 [==============================] - 8s 837ms/step - loss: 681.9034 - mse_known: 0.1210 - mse_unknown: 0.1758 - lr: 5.0

10/10 [==============================] - 9s 892ms/step - loss: 659.9474 - mse_known: 0.1279 - mse_unknown: 0.1839 - lr: 4.2196e-04
Epoch 33/100
10/10 [==============================] - 8s 845ms/step - loss: 659.4022 - mse_known: 0.1278 - mse_unknown: 0.1838 - lr: 4.0573e-04
Epoch 34/100
10/10 [==============================] - 8s 841ms/step - loss: 658.8600 - mse_known: 0.1277 - mse_unknown: 0.1838 - lr: 3.9012e-04
Epoch 35/100
10/10 [==============================] - 8s 842ms/step - loss: 658.3398 - mse_known: 0.1276 - mse_unknown: 0.1837 - lr: 3.7512e-04
Epoch 36/100
10/10 [==============================] - 8s 838ms/step - loss: 657.8566 - mse_known: 0.1275 - mse_unknown: 0.1836 - lr: 3.6069e-04
Epoch 37/100
10/10 [==============================] - 8s 833ms/step - loss: 657.4250 - mse_known: 0.1274 - mse_unknown: 0.1836 - lr: 3.4682e-04
Epoch 38/100
10/10 [==============================] - 8s 849ms/step - loss: 656.9474 - mse_known: 0.1273 - mse_unknown: 0.1835 - lr: 3.3348e-04
Epoch

Epoch 89/100
10/10 [==============================] - 9s 893ms/step - loss: 646.6872 - mse_known: 0.1253 - mse_unknown: 0.1823 - lr: 4.5120e-05
Epoch 90/100
10/10 [==============================] - 8s 851ms/step - loss: 646.6634 - mse_known: 0.1253 - mse_unknown: 0.1823 - lr: 4.3384e-05
Epoch 91/100
10/10 [==============================] - 8s 847ms/step - loss: 646.5599 - mse_known: 0.1253 - mse_unknown: 0.1823 - lr: 4.1716e-05
Epoch 92/100
10/10 [==============================] - 8s 850ms/step - loss: 646.5212 - mse_known: 0.1253 - mse_unknown: 0.1823 - lr: 4.0111e-05
Epoch 93/100
10/10 [==============================] - 8s 852ms/step - loss: 646.4694 - mse_known: 0.1253 - mse_unknown: 0.1822 - lr: 3.8568e-05
Epoch 94/100
10/10 [==============================] - 9s 855ms/step - loss: 646.4315 - mse_known: 0.1253 - mse_unknown: 0.1822 - lr: 3.7085e-05
Epoch 95/100
10/10 [==============================] - 8s 854ms/step - loss: 646.3967 - mse_known: 0.1253 - mse_unknown: 0.1822 - lr: 3.5

In [16]:
for i, rem in enumerate(sensing_ratio):
    print(f"For {int(100 * rem)}% sensing ratio: {learned_variables[i]}")

For 1% sensing ratio: [19.989794, 19.96376, 19.86665, 19.50292]
For 5% sensing ratio: [19.991055, 19.967804, 19.88245, 19.565823]
For 10% sensing ratio: [19.992077, 19.97209, 19.90004, 19.635563]
For 15% sensing ratio: [19.993273, 19.976612, 19.917591, 19.703789]
For 20% sensing ratio: [19.994295, 19.98038, 19.932253, 19.759691]
For 25% sensing ratio: [19.994865, 19.98352, 19.94417, 19.8047]
For 30% sensing ratio: [19.995539, 19.986647, 19.955816, 19.848183]
For 35% sensing ratio: [19.9961, 19.989113, 19.965137, 19.882336]
For 40% sensing ratio: [19.996586, 19.991533, 19.973606, 19.91283]
For 45% sensing ratio: [19.99744, 19.993765, 19.9807, 19.937561]
For 50% sensing ratio: [19.998373, 19.995155, 19.98606, 19.956175]


In [17]:
np.save('./Outputs/eldar_chepuri_mse_known.npy', mse_known_list)
np.save('./Outputs/eldar_chepuri_mse_unknown.npy', mse_unknown_list)